In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)


In [3]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/fraudTest.csv')
df.head()

Mounted at /content/drive


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [4]:
df = df.sample(n=100_000, random_state=42)


In [5]:
df = df.drop(columns=[
    'Unnamed: 0',
    'cc_num',
    'first',
    'last',
    'street',
    'merchant',
    'city',
    'job',
    'trans_num',
    'trans_date_trans_time',
    'dob'
], errors='ignore')


In [6]:
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']


In [7]:
print(y.isna().sum())


0


In [8]:
X = pd.get_dummies(
    X,
    columns=['category', 'gender', 'state'],
    drop_first=True
)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
rf = RandomForestClassifier(
    n_estimators=50,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=50, n_jobs=-1,
                       random_state=42)

In [11]:
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]


In [12]:
print("Random Forest – Fraud Detection Report")
print(classification_report(y_test, y_pred))


Random Forest – Fraud Detection Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19923
           1       0.96      0.35      0.51        77

    accuracy                           1.00     20000
   macro avg       0.98      0.68      0.76     20000
weighted avg       1.00      1.00      1.00     20000



In [13]:
performance_table = pd.DataFrame({
    "Metric": ["Precision (Fraud)", "Recall (Fraud)", "F1-Score (Fraud)", "ROC-AUC"],
    "Score": [
        precision_score(y_test, y_pred),
        recall_score(y_test, y_pred),
        f1_score(y_test, y_pred),
        roc_auc_score(y_test, y_prob)
    ]
})

performance_table


,Metric,Score
0,Precision (Fraud),0.964286
1,Recall (Fraud),0.350649
2,F1-Score (Fraud),0.514286
3,ROC-AUC,0.929875


In [14]:
cm = confusion_matrix(y_test, y_pred)

cm_table = pd.DataFrame(
    cm,
    index=["Actual Legit", "Actual Fraud"],
    columns=["Predicted Legit", "Predicted Fraud"]
)

cm_table


,Predicted Legit,Predicted Fraud
Actual Legit,19922,1
Actual Fraud,50,27


In [15]:
sample_predictions = pd.DataFrame({
    "Actual": y_test.values[:10],
    "Predicted": y_pred[:10]
})

sample_predictions


,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,0,0
7,0,0
8,0,0
9,0,0
